## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("cities_description.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Bethel,41.37,-73.41,75.99,83,1,1.99,US,2020-08-22 14:09:54,clear sky
1,1,Rikitea,-23.12,-134.97,66.13,71,100,19.55,PF,2020-08-22 14:17:26,overcast clouds
2,2,Saint-Philippe,-21.36,55.77,71.60,68,35,19.46,RE,2020-08-22 14:17:57,scattered clouds
3,3,Victoria,22.29,114.16,84.99,75,100,1.99,HK,2020-08-22 14:18:56,overcast clouds
4,4,Kapaa,22.08,-159.32,78.80,74,1,13.87,US,2020-08-22 14:21:40,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
personalized_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
personalized_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Bethel,41.37,-73.41,75.99,83,1,1.99,US,2020-08-22 14:09:54,clear sky
1,1,Rikitea,-23.12,-134.97,66.13,71,100,19.55,PF,2020-08-22 14:17:26,overcast clouds
2,2,Saint-Philippe,-21.36,55.77,71.60,68,35,19.46,RE,2020-08-22 14:17:57,scattered clouds
3,3,Victoria,22.29,114.16,84.99,75,100,1.99,HK,2020-08-22 14:18:56,overcast clouds
4,4,Kapaa,22.08,-159.32,78.80,74,1,13.87,US,2020-08-22 14:21:40,clear sky
5,5,Airai,-8.93,125.41,57.78,65,30,4.07,TL,2020-08-22 14:21:40,scattered clouds
6,6,Hamilton,39.18,-84.53,75.99,83,75,3.36,US,2020-08-22 14:21:40,light rain
7,7,San Patricio,19.22,-104.70,80.47,82,4,2.24,MX,2020-08-22 14:17:26,clear sky
9,9,Bredasdorp,-34.53,20.04,68.00,60,0,12.75,ZA,2020-08-22 14:21:41,clear sky
10,10,Danjiangkou,32.54,111.51,72.36,93,97,1.34,CN,2020-08-22 14:21:41,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
personalized_cities_df.count()

City_ID                583
City                   583
Lat                    583
Lng                    583
Max Temp               583
Humidity               583
Cloudiness             583
Wind Speed             583
Country                577
Date                   583
Current Description    583
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Based on the above, there are 6 empty values in the country column, and therefore 
# we should drop 6 rows and end with a total of 577 rows instead of 583
clean_df = personalized_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Bethel,41.37,-73.41,75.99,83,1,1.99,US,2020-08-22 14:09:54,clear sky
1,1,Rikitea,-23.12,-134.97,66.13,71,100,19.55,PF,2020-08-22 14:17:26,overcast clouds
2,2,Saint-Philippe,-21.36,55.77,71.60,68,35,19.46,RE,2020-08-22 14:17:57,scattered clouds
3,3,Victoria,22.29,114.16,84.99,75,100,1.99,HK,2020-08-22 14:18:56,overcast clouds
4,4,Kapaa,22.08,-159.32,78.80,74,1,13.87,US,2020-08-22 14:21:40,clear sky
...,...,...,...,...,...,...,...,...,...,...,...
696,696,Copiapo,-27.37,-70.33,53.60,71,90,9.17,CL,2020-08-22 14:22:56,overcast clouds
698,698,Wilmington,34.23,-77.94,84.00,69,1,8.05,US,2020-08-22 14:22:56,clear sky
699,699,Tornio,65.85,24.15,66.20,82,75,12.75,FI,2020-08-22 14:22:56,broken clouds
700,700,Santo Domingo,18.50,-69.99,86.00,70,40,18.34,DO,2020-08-22 14:22:57,scattered clouds


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,75.99,clear sky,41.37,-73.41,
1,Rikitea,PF,66.13,overcast clouds,-23.12,-134.97,
2,Saint-Philippe,RE,71.60,scattered clouds,-21.36,55.77,
3,Victoria,HK,84.99,overcast clouds,22.29,114.16,
4,Kapaa,US,78.80,clear sky,22.08,-159.32,
5,Airai,TL,57.78,scattered clouds,-8.93,125.41,
6,Hamilton,US,75.99,light rain,39.18,-84.53,
7,San Patricio,MX,80.47,clear sky,19.22,-104.70,
9,Bredasdorp,ZA,68.00,clear sky,-34.53,20.04,
10,Danjiangkou,CN,72.36,overcast clouds,32.54,111.51,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame & add the latitude and longitude to location key for the 
    # params dictionary
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,75.99,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
1,Rikitea,PF,66.13,overcast clouds,-23.12,-134.97,Pension Maro'i
2,Saint-Philippe,RE,71.60,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
3,Victoria,HK,84.99,overcast clouds,22.29,114.16,Mini Hotel Central
4,Kapaa,US,78.80,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
696,Copiapo,CL,53.60,overcast clouds,-27.37,-70.33,Antay Casino Hotel
698,Wilmington,US,84.00,clear sky,34.23,-77.94,"Hotel Ballast Wilmington, Tapestry Collection ..."
699,Tornio,FI,66.20,broken clouds,65.85,24.15,Park Hotel Tornio
700,Santo Domingo,DO,86.00,scattered clouds,18.50,-69.99,Micro Hotel Condo Suites


In [32]:
# 7a. Identify if or how many rows have empty values for hotels
hotel_df.count()

City                   577
Country                577
Max Temp               577
Current Description    577
Lat                    577
Lng                    577
Hotel Name             577
dtype: int64

In [45]:
# 7b. Although there are no null values, they may have empty strings.  Convert these to NaN.
hotel_df.loc[(hotel_df["Hotel Name"] == "")]= np.nan

In [46]:
# 7c. Drop the rows where there are NaN values where there were once empty strings.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bethel,US,75.99,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
1,Rikitea,PF,66.13,overcast clouds,-23.12,-134.97,Pension Maro'i
2,Saint-Philippe,RE,71.60,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
3,Victoria,HK,84.99,overcast clouds,22.29,114.16,Mini Hotel Central
4,Kapaa,US,78.80,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
5,Airai,TL,57.78,scattered clouds,-8.93,125.41,Juvinal Martin
6,Hamilton,US,75.99,light rain,39.18,-84.53,Six Acres Bed & Breakfast
7,San Patricio,MX,80.47,clear sky,19.22,-104.70,Hotel Cabo Blanco
9,Bredasdorp,ZA,68.00,clear sky,-34.53,20.04,Bredasdorp Country Manor
10,Danjiangkou,CN,72.36,overcast clouds,32.54,111.51,Dongsheng Guesthouse


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and max temp of {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))